# Exercise 3.2

Import the pySpark libraries

In [6]:
import sys
!conda install --yes --prefix {sys.prefix} -c conda-forge pyspark

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.7.12
  latest version: 4.8.2

Please update conda by running

    $ conda update -n base conda



## Package Plan ##

  environment location: /home/bas/anaconda3

  added / updated specs:
    - pyspark


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    py4j-0.10.7                |             py_1         177 KB  conda-forge
    pyspark-2.4.5              |             py_0       207.0 MB  conda-forge
    ------------------------------------------------------------
                                           Total:       207.2 MB

The following NEW packages will be INSTALLED:

  py4j               conda-forge/noarch::py4j-0.10.7-py_1
  pyspark            conda-forge/noarch::pyspark-2.4.5-py_0



py4j-0.10.7          | 177 KB    | ##################################### | 100% 
pyspark-2.4.5    

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, split, size

Connect to a local Spark session

In [ ]:
spark = SparkSession.builder.appName("Packt").getOrCreate()

Read the raw data from a CSV file

In [ ]:
data = spark.read.csv('../../Datasets/netflix_titles_nov_2019.csv', header='true')

Take only the movies

In [ ]:
movies = data.filter((col('type') == 'Movie') & (col('release_year') == 2019))

Add a column with the number of actors

In [ ]:
transformed = movies.withColumn('count_cast', size(split(movies['cast'], ',')))

Select a subset of columns to store

In [ ]:
selected = transformed.select('title', 'director', 'count_cast', 'cast', 'rating', 'release_year', 'type')

Write the contents of the DataFrame to disk

In [ ]:
selected.write.csv('transformed.csv', header='true')